In [97]:
import flow
from flow.networks.ring import RingNetwork
from flow.core.params import VehicleParams
from flow.controllers.car_following_models import IDMController
from flow.controllers.routing_controllers import ContinuousRouter
from flow.envs.ring.accel import AccelEnv
from flow.core.experiment import Experiment


In [98]:
from flow.controllers.base_lane_changing_controller import BaseLaneChangeController

# We want the ambulance to always prefer the left lane, so we create this lane-changing controller that makes them always enter the left lane
class LeftLaneController(BaseLaneChangeController):
    """A lane-changing model used to move vehicles into the left lane."""
    ##### Below this is new code #####
    def get_lane_change_action(self, env):
        current_lane = env.k.vehicle.get_lane(self.veh_id)
        if current_lane < 4:
            return 1
        else:
            return 0

In [99]:

vehicles = VehicleParams()
vehicles.add("ambulance",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(LeftLaneController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=1,
             color="red")
# vehicles.add("human",
#              acceleration_controller=(IDMController, {}),
#              routing_controller=(ContinuousRouter, {}),
#              num_vehicles=22)

In [100]:
from flow.networks.ring import ADDITIONAL_NET_PARAMS
from flow.core.params import NetParams

net_params = NetParams(additional_params={**ADDITIONAL_NET_PARAMS, "lanes": 4})

In [101]:
from flow.core.params import InitialConfig

initial_config = InitialConfig(spacing="uniform", perturbation=1)

In [102]:
from flow.core.params import TrafficLightParams

traffic_lights = TrafficLightParams()

In [103]:
from flow.core.params import SumoParams

sim_params = SumoParams(sim_step=0.1, render=True, emission_path='data')

In [104]:
from flow.envs.ring.accel import ADDITIONAL_ENV_PARAMS
from flow.core.params import EnvParams

env_params = EnvParams(additional_params={**ADDITIONAL_ENV_PARAMS, "horizon": 3000})

In [105]:
flow_params = dict(
    exp_tag='ambulance',
    env_name=AccelEnv,
    network=RingNetwork,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
    tls=traffic_lights,
)

# number of time steps
flow_params['env'].horizon = 3000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1, convert_to_csv=True)

Round 0, return: 102.93383751210014
data/ambulance_20220827-1336101661621770.298505-0_emission.csv data
Average, std returns: 102.93383751210014, 0.0
Average, std velocities: 28.30231917626482, 0.0
Average, std outflows: 0.0, 0.0
Total time: 11.083554983139038
steps/second: 2121.4451791457313
